In [47]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.TwoDoF as TwoDof
import FwtModels as fm

from matplotlib.lines import Line2D

me.mechanics_printing()

In [49]:
half = sym.Rational(1,2)
panels = 10
p = TwoDof.base_params(panels)

In [50]:
#Define refernce frames
wing_root_frame = mf.HomogenousTransform().Translate(0,0,p.q[0]).R_x(-p.q[1])
wing_aero_frame = wing_root_frame.Translate(0,p.y_0,0)
wing_com_frame = wing_root_frame.Translate(0,p.l_com,0)

#Define Mass Matrices
M_fwt = ele.MassMatrix(p.m,I_xx = 0)

# Define Elements
M_innerWing = ele.RigidElement.PointMass(wing_root_frame,p.m_w,True)
fwt_ele = ele.RigidElement(wing_com_frame,M_fwt,True)
spring_ele = ele.Spring(p.q[0],p.k_w)

# Create AeroForces
wing_AeroForces = ef.AeroForce.PerUnitSpan(p,wing_aero_frame,p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = p.alpha_1,
                               alpha_zero = p.alpha_r,
                               stall_angle = 0,
                               c = p.c_root,
                               c_d_max = 0,
                               linear = True)
# Split FWT into Segments
forces = []
for i in range(panels):
    seg_width = p.s/panels
    yi = seg_width/2 + i*seg_width
    forces.append(wing_AeroForces.subs({p.y_0:yi,p.a_0:p.a[i]})*seg_width)
Q = sym.Matrix([0]*p.qs)
for f in forces:
    Q += f.Q()
c_forces = ef.ExternalForce(Q)

In [51]:
#sm = mf.SymbolicModel.FromElementsAndForces(p,[M_innerWing,fwt_ele,spring_ele,spring_ele_fwt],c_forces)
sm = mf.SymbolicModel.FromElementsAndForces(p,[M_innerWing,fwt_ele,spring_ele],c_forces)

In [52]:
sm.to_file('2Dof_Model.py')

In [37]:
sm.ExtForces.Q()[1]

In [31]:
wing_root_frame.Transform_point([0,0,0])

⎡0 ⎤
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎣q₀⎦